In [3]:
pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:
pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.3 MB/s eta 0:00:00


In [4]:
from faker import Faker
import random
import torch
from transformers import DistilBertTokenizer, DistilBertConfig,DistilBertForSequenceClassification,AdamW
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm  # Import tqdm for the loading bar


In [5]:
# Check if a GPU is available
if torch.cuda.is_available():
    # Get the name of the GPU
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU: {gpu_name}")
else:
    print("No GPU available.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU: Tesla V100-SXM2-16GB


In [ ]:
def generate_two_worded(x,y):
    # Generate two random words
    word1 = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=random.randint(x,y)))
    word2 = ''.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=random.randint(3, 10)))

    # Concatenate the two words with a space in between
    two_worded_string = f"{word1} {word2}"

    return two_worded_string

In [ ]:
# Initialize the Faker instance
faker = Faker()

# Generate fake names and random words
num_fake_names = 500000
fake_nameandsurname = [faker.name() for _ in range(num_fake_names)]
fake_name = [faker.first_name() for _ in range(num_fake_names)]
fake_names = fake_nameandsurname + fake_name
two_worded_strings = [generate_two_worded(3,10) for _ in range(300000)]
two_worded_strings2 = [generate_two_worded(5,8) for _ in range(200000)]

single_random_words = [''.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=random.randint(3, 10))) for _ in range(300000)]

single_random_words2 = [''.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=random.randint(5, 8))) for _ in range(200000)]


random_words = two_worded_strings + two_worded_strings2 + single_random_words + single_random_words2

random_words = list(set(random_words))

# Combine fake names and random words
all_names = fake_names + random_words



# Label the data (1 for valid names, 0 for invalid names)
labels = [1] * len(fake_names) + [0] * len(random_words)

# Shuffle the data
data = list(zip(all_names, labels))
random.shuffle(data)
names, labels = zip(*data)

# Example: Display the first 10 names and labels
for name, label in zip(names[:10], labels[:10]):
    print(f"Name: {name}, Label: {label}")

Name: dsqhxla vzkocksdr, Label: 0
Name: Kathleen Castillo, Label: 1
Name: Jennifer Edwards, Label: 1
Name: ztwozi dcuzw, Label: 0
Name: kmwdust, Label: 0
Name: sgeaqsz, Label: 0
Name: xpth, Label: 0
Name: iylwxtkfa, Label: 0
Name: gjgbryvt hhscqypjm, Label: 0
Name: Matthew Martinez, Label: 1


In [ ]:
print(len(labels))

1975856


In [ ]:
print(len(all_names))

1975856


In [ ]:
# Step 2: Load Pre-trained DistilBERT Model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
inputs = tokenizer(names, return_tensors='pt', padding=True, truncation=True).to(device)

In [ ]:
# Step 5: Fine-tune the Model
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], torch.tensor(labels))
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
train_names, val_names, train_labels, val_labels = train_test_split(names, labels, test_size=0.2, random_state=42)

In [ ]:
# Tokenize validation set
val_inputs = tokenizer(val_names, return_tensors='pt', padding=True, truncation=True).to(device)
val_dataset = TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], torch.tensor(val_labels))
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False)  # No need to shuffle for validation

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)  # You can experiment with different learning rates

num_epochs = 4  # Adjust as needed

# Training Loop with Validation
for epoch in range(num_epochs):
    model.train()

    tqdm_dataloader = tqdm(dataloader, desc=f"Epoch {epoch + 1}", position=0, leave=True)

    for batch in tqdm_dataloader:
        input_ids, attention_mask, target_labels = batch

        optimizer.zero_grad()
        outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device), labels=target_labels.to(device))
        loss = outputs.loss
        logits = outputs.logits  # Ensure we are correctly accessing logits
        loss.backward()
        optimizer.step()

        tqdm_dataloader.set_postfix({"Loss": loss.item()}, refresh=True)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0

    with torch.no_grad():
        for val_batch in val_dataloader:
            val_input_ids, val_attention_mask, val_target_labels = val_batch
            val_outputs = model(val_input_ids.to(device), attention_mask=val_attention_mask.to(device), labels=val_target_labels.to(device))
            val_loss += val_outputs.loss.item()
            val_logits = val_outputs.logits  # Ensure we are correctly accessing logits

            # Calculate accuracy on validation set
            predictions = torch.argmax(val_logits, dim=1)
            val_correct += (predictions == val_target_labels.to(device)).sum().item()

    val_loss /= len(val_dataloader)
    val_accuracy = val_correct / len(val_dataset)

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")


Epoch 1: 100%|██████████| 15437/15437 [17:54<00:00, 14.37it/s, Loss=1.13e-5]


Epoch 1, Loss: 1.1339267985022161e-05, Validation Loss: 0.00021116300555606895, Validation Accuracy: 0.99997469456338


Epoch 2: 100%|██████████| 15437/15437 [17:54<00:00, 14.37it/s, Loss=2.49e-6]


Epoch 2, Loss: 2.4884811864467338e-06, Validation Loss: 0.0002041837445798793, Validation Accuracy: 0.99997469456338


Epoch 3: 100%|██████████| 15437/15437 [17:53<00:00, 14.38it/s, Loss=1.92e-5]


Epoch 3, Loss: 1.9197623259969987e-05, Validation Loss: 0.00018694538406623185, Validation Accuracy: 0.99997469456338


Epoch 4: 100%|██████████| 15437/15437 [17:52<00:00, 14.39it/s, Loss=8.81e-6]


Epoch 4, Loss: 8.811227417027112e-06, Validation Loss: 0.00017609884513983251, Validation Accuracy: 0.999979755650704


In [ ]:
# Step 6: Inference
new_names = ["Ellis","Ellis Miles","Ivy William","dsagfsdx afdsgas","asdas asdas","Jane Eyre","Sarah Benson"]
new_inputs = tokenizer(new_names, return_tensors='pt', padding=True, truncation=True)

with torch.no_grad():
    model.eval()
    outputs = model(**new_inputs.to(device))

predictions = torch.argmax(outputs.logits, dim=1).tolist()
print("Predictions for new names:", predictions)

Predictions for new names: [0, 1, 1, 0, 0, 1, 1]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Assuming model is your DistilBERT model
model.save_pretrained("/content/drive/My Drive/distilbert_model_ENG")
tokenizer.save_pretrained("/content/drive/My Drive/distilbert_model_ENG")

('/content/drive/My Drive/distilbert_model_ENG/tokenizer_config.json',
 '/content/drive/My Drive/distilbert_model_ENG/special_tokens_map.json',
 '/content/drive/My Drive/distilbert_model_ENG/vocab.txt',
 '/content/drive/My Drive/distilbert_model_ENG/added_tokens.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the model and tokenizer
model_path = '/content/drive/My Drive/distilbert_model'
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

In [ ]:
# Now you can use the model and tokenizer for inference or further training
text = "assgsa sdafdsfgh"
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

# Convert probabilities to binary predictions with a 0.5 threshold
predictions = torch.argmax(logits, dim=1).tolist()

print(predictions)  # This will print a tensor of 0s and 1s


[0]


In [ ]:
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
def load_data_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        names = []
        labels = []
        for line in lines:
          try:
            name, label = line.strip().split(', ')
            names.append(name)
            labels.append(int(label))
          except:
            continue
    return names, labels

In [ ]:
# Load your data
file_path1 = '/content/drive/My Drive/classified_words.txt'  # Replace with your file path
names1, labels1 = load_data_from_txt(file_path1)

In [ ]:

file_path2 = '/content/drive/My Drive/classified_namess_meaningful.txt'  # Replace with your file path
names2, labels2 = load_data_from_txt(file_path2)

In [ ]:
names = names1 + names2
labels = labels1 + labels2

In [ ]:
# Tokenize the data
encodings = tokenizer(names, truncation=True, padding=True, return_tensors="pt")

In [ ]:
# Step 5: Fine-tune the Model
dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], torch.tensor(labels))
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)  # You can experiment with different learning rates

num_epochs = 4  # Adjust as needed

# Training Loop with Validation
for epoch in range(num_epochs):
    model.train()

    tqdm_dataloader = tqdm(dataloader, desc=f"Epoch {epoch + 1}", position=0, leave=True)

    for batch in tqdm_dataloader:
        input_ids, attention_mask, target_labels = batch

        optimizer.zero_grad()
        outputs = model(input_ids.to(device), attention_mask=attention_mask.to(device), labels=target_labels.to(device))
        loss = outputs.loss
        logits = outputs.logits  # Ensure we are correctly accessing logits
        loss.backward()
        optimizer.step()

        tqdm_dataloader.set_postfix({"Loss": loss.item()}, refresh=True)


    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1: 100%|██████████| 1993/1993 [11:25<00:00,  2.91it/s, Loss=0.00103]


Epoch 1, Loss: 0.0010328374337404966


Epoch 2: 100%|██████████| 1993/1993 [11:26<00:00,  2.90it/s, Loss=0.000733]


Epoch 2, Loss: 0.0007333785761147738


Epoch 3: 100%|██████████| 1993/1993 [11:26<00:00,  2.90it/s, Loss=0.000457]


Epoch 3, Loss: 0.0004570462042465806


Epoch 4: 100%|██████████| 1993/1993 [11:26<00:00,  2.90it/s, Loss=0.000321]

Epoch 4, Loss: 0.0003207257541362196


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Assuming model is your DistilBERT model
model.save_pretrained("/content/drive/My Drive/distilbert_model_ENG_2")
tokenizer.save_pretrained("/content/drive/My Drive/distilbert_model_ENG_2")

('/content/drive/My Drive/distilbert_model_ENG_2/tokenizer_config.json',
 '/content/drive/My Drive/distilbert_model_ENG_2/special_tokens_map.json',
 '/content/drive/My Drive/distilbert_model_ENG_2/vocab.txt',
 '/content/drive/My Drive/distilbert_model_ENG_2/added_tokens.json')

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import random
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertConfig,DistilBertForSequenceClassification,AdamW
from torch.utils.data import TensorDataset, DataLoader
# Load the model and tokenizer
model_path = "/content/drive/My Drive/distilbert_model_ENG_2"
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

model.eval()
model.to(device)
# Load the CSV file
csv_path = r"/content/drive/My Drive/200K USA citizenship database.csv"  # Update this to the path of your CSV file
df = pd.read_csv(csv_path, sep=';')

# Select the names column (update 'name_column' to your actual column name with names)
names = df["Name"].tolist()

# Output file
output_file_path = 'classified_names.txt'

# Open a file to write the classification results
with open(output_file_path, 'w') as output_file:
    # Classify each name
    for name in names:
        # Tokenize the name
        inputs = tokenizer(name, return_tensors="pt", padding=True, truncation=True).to(device)

        # Get the model's logits
        with torch.no_grad():
            logits = model(**inputs).logits

        # Convert logits to probabilities and then to binary predictions
        predictions = torch.argmax(logits, dim=1).tolist()

        # Write the name and its classification to the text file
        classification = 'random' if predictions[0] == 0 else 'not random'
        output_file.write(f"{name}: {classification}\n")

print(f"Classification complete. Results saved to {output_file_path}")

Classification complete. Results saved to classified_names.txt


In [10]:
import random
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertConfig,DistilBertForSequenceClassification,AdamW
from torch.utils.data import TensorDataset, DataLoader
# Load the model and tokenizer
model_path = "/content/drive/My Drive/bitirme_ENG/distilbert_model_ENG_2"
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

model.eval()

# Load the CSV file
csv_path = r"/content/drive/My Drive/bitirme_ES/jasmin.rs.csv"  # Update this to the path of your CSV file
df = pd.read_csv(csv_path, sep=';')

# Select the names column (update 'name_column' to your actual column name with names)
names = df["Name"].tolist()

# Output file
output_file_path = 'classified_namess.txt'

# Open a file to write the classification results
with open(output_file_path, 'w') as output_file:
    # Classify each name
    for name in names:
        # Tokenize the name
        inputs = tokenizer(name, return_tensors="pt", padding=True, truncation=True)

        # Get the model's logits
        with torch.no_grad():
            logits = model(**inputs).logits

        # Convert logits to probabilities and then to binary predictions
        predictions = torch.argmax(logits, dim=1).tolist()

        # Write the name and its classification to the text file
        classification = 'random' if predictions[0] == 0 else 'not random'
        output_file.write(f"{name}: {classification}\n")

print(f"Classification complete. Results saved to {output_file_path}")

KeyboardInterrupt: 

In [11]:
!pip freeze > requirements.txt

In [12]:
import random
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertConfig,DistilBertForSequenceClassification,AdamW
from torch.utils.data import TensorDataset, DataLoader
# Load the model and tokenizer
model_path = "/content/drive/My Drive/bitirme_ENG/distilbert_model_ENG_2"
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)


In [13]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Assuming model is your DistilBERT model
model.save_pretrained("distilbert_model")
tokenizer.save_pretrained("distilbert_model")

('distilbert_model/tokenizer_config.json',
 'distilbert_model/special_tokens_map.json',
 'distilbert_model/vocab.txt',
 'distilbert_model/added_tokens.json')

In [14]:
# Load the model and tokenizer
model_path = 'distilbert_model'
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)